In [5]:

import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import re
import os
from scipy.spatial.transform import Rotation as R


In [8]:
def parse_data(path: str):
    files = glob.glob(f'{path}*[0-9].txt')
    files = sorted(files, key=lambda x:int(re.findall("(\d+)",x)[-1]))

    poses = np.zeros((len(files),6))
    coords = np.zeros((len(files),3))
    angles = np.zeros((len(files),3))


    for i, f in enumerate(files):
        
    #print(np.loadtxt(i))
        with open(f, 'r') as f_input:
            lines = f_input.readlines()
            
        # poses[i] = []
        numbers = []
        for line in lines:
            m = re.sub(r'\[|\]' ,'', line)
            m = [float(s) for s in m.split()]
            numbers += m 
        poses[i] = numbers
        
    coords = poses[:,:3]
    angles = poses[:,3:]

    return angles, coords

In [9]:
def get_rot_matrix_from_euler(angles):
    # R = np.array(angles)
    # R, J = cv2.Rodrigues(R)
    
    tx = angles[0]
    ty = angles[1] 
    tz = angles[2]
    
    Rx = [[1, 0, 0], [0, np.cos(tx), -np.sin(tx)], [0, np.sin(tx), np.cos(tx)]]
    Ry = [[np.cos(ty), 0, np.sin(ty)], [0, 1, 0], [-np.sin(ty), 0, np.cos(ty)]]
    Rz = [[np.cos(tz), -np.sin(tz), 0], [np.sin(tz), np.cos(tz), 0], [0, 0, 1]]
    R = np.matmul(Rz, np.matmul(Ry, Rx))
    return R

def get_rot_matrix_from_euler_nipun(self, angles:list):
    Rx = np.array([[1,                   0,                  0],
                    [0,   np.cos(angles[0]), -np.sin(angles[0])],
                    [0,   np.sin(angles[0]),  np.cos(angles[0])]])

    Ry = np.array([[ np.cos(angles[1]),    0,   np.sin(angles[1])],
                    [                 0,    1,                   0],
                    [-np.sin(angles[1]),    0,   np.cos(angles[1])]])

    Rz = np.array([[np.cos(angles[2]),  -np.sin(angles[2]), 0],
                    [np.sin(angles[2]),   np.cos(angles[2]), 0],
                    [                0,                   0, 1]])

    rot = Rx@Ry@Rz 
    return rot

def get_translation(coord):

    offset_base_to_rgb = np.array([10, 32, 13])/1000
    #From vicon link to base link
    
    T = np.array(coord)
    T = T + offset_base_to_rgb
    T = T*10**(-6)
    return T

def get_intrinsic(fx, fy, cx, cy):
    return np.array([[fx, 0 , cx],
                       [0, fy, cy],
                       [0, 0, 1]])

In [10]:
def main():
    
    transform = np.zeros((4,4))

    # for realsense D435i
    # TODO: load it from .yaml/.json file
    fx = 615.671 # focal dist of cam lense [mm]
    fy = 615.962 # 
    cx = 328.001 # optical center
    cy = 241.31

    outdir = '/home/iana/Atlas/drone_view/'
    indir = '/home/iana/Atlas/drone_view/poses_raw/'

    angles = parse_data(indir)[0] # get raw angle data
    coord = parse_data(indir)[1] # get raw coord
    
    # TODO: change it to Nipun's rotations
    tx = -90*(np.pi/180) # -90
    ty = 0.776*(np.pi/180) # 0.776
    tz = -90*(np.pi/180) # -90
    
    Rx = [[1, 0, 0], [0, np.cos(tx), -np.sin(tx)], [0, np.sin(tx), np.cos(tx)]]
    Ry = [[np.cos(ty), 0, np.sin(ty)], [0, 1, 0], [-np.sin(ty), 0, np.cos(ty)]]
    Rz = [[np.cos(tz), -np.sin(tz), 0], [np.sin(tz), np.cos(tz), 0], [0, 0, 1]]
    #R_base_to_camera_color_opt = get_rot((tx, ty, tz))
    
    for i, (ang, c) in enumerate(zip(angles, coord)):
    
        R = get_rot_matrix_from_euler(ang)
        R = np.matmul(Rz, np.matmul(Ry, np.matmul(Rx, R)))
        #R = R_base_to_camera_color_opt @ R

        transform[0:3, 0:3] = R
       

        T = get_translation(c)
        transform[0:3, 3] = T
        transform[3, 3] = 1
        np.savetxt(f'{outdir}pose/{i+1:08}.txt', transform)
    
    cam_param= get_intrinsic(fx, fy, cx, cy)
    np.savetxt(f'{outdir}intrinsics.txt', cam_param)

main()

In [13]:
from scipy.spatial.transform import Rotation as R
transform = np.zeros((4,4))

r = R.from_rotvec([0, 0, np.pi/2])

transform[0:3, 0:3] = r.as_matrix()
print(transform, r.as_matrix())

[[ 2.22044605e-16 -1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  2.22044605e-16  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] [[ 2.22044605e-16 -1.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  2.22044605e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [ ]:
outdir = '/home/iana/Atlas/drone_view/color/'


for i, f in enumerate(os.listdir(outdir)):

    old_name = f'{outdir}{i+1:07}.jpg'
    new_name = f'{outdir}{i+1:08}.jpg'

    # Renaming the file
    os.rename(old_name, new_name)

In [9]:
import numpy as np
coords = np.zeros((10,3))
coords[0] = 1

print(coords)

x_min = coords[0].min(axis=1)
y_min = coords[1].min(axis=1)
coords[0] = coords[0] - x_min
coords[1] = coords[1] - y_min

coords

[[1. 1. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


AxisError: axis 1 is out of bounds for array of dimension 1